<a href="https://colab.research.google.com/github/ddodds42/DS-Unit-2-Linear-Models/blob/master/module3-ridge-regression/DAVID_DODDS_LS_DS_213_assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Lambda School Data Science

*Unit 2, Sprint 1, Module 3*

---

In [0]:
%%capture
import sys

# If you're on Colab:
if 'google.colab' in sys.modules:
    DATA_PATH = 'https://raw.githubusercontent.com/LambdaSchool/DS-Unit-2-Applied-Modeling/master/data/'
    !pip install category_encoders==2.*

# If you're working locally:
else:
    DATA_PATH = '../data/'
    
# Ignore this Numpy warning when using Plotly Express:
# FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
import warnings
warnings.filterwarnings(action='ignore', category=FutureWarning, module='numpy')

In [0]:
import pandas as pd
import pandas_profiling

# Read New York City property sales data
df = pd.read_csv(DATA_PATH+'condos/NYC_Citywide_Rolling_Calendar_Sales.csv')

# Change column names: replace spaces with underscores
df.columns = [col.replace(' ', '_') for col in df]

# SALE_PRICE was read as strings.
# Remove symbols, convert to integer
df['SALE_PRICE'] = (
    df['SALE_PRICE']
    .str.replace('$','')
    .str.replace('-','')
    .str.replace(',','')
    .astype(int)
)

In [0]:
# BOROUGH is a numeric column, but arguably should be a categorical feature,
# so convert it from a number to a string
df['BOROUGH'] = df['BOROUGH'].astype(str)

In [0]:
# Reduce cardinality for NEIGHBORHOOD feature

# Get a list of the top 10 neighborhoods
top10 = df['NEIGHBORHOOD'].value_counts()[:10].index

# At locations where the neighborhood is NOT in the top 10, 
# replace the neighborhood with 'OTHER'
df.loc[~df['NEIGHBORHOOD'].isin(top10), 'NEIGHBORHOOD'] = 'OTHER'

In [389]:
df.head()

,BOROUGH,NEIGHBORHOOD,BUILDING_CLASS_CATEGORY,TAX_CLASS_AT_PRESENT,BLOCK,LOT,EASE-MENT,BUILDING_CLASS_AT_PRESENT,ADDRESS,APARTMENT_NUMBER,ZIP_CODE,RESIDENTIAL_UNITS,COMMERCIAL_UNITS,TOTAL_UNITS,LAND_SQUARE_FEET,GROSS_SQUARE_FEET,YEAR_BUILT,TAX_CLASS_AT_TIME_OF_SALE,BUILDING_CLASS_AT_TIME_OF_SALE,SALE_PRICE,SALE_DATE
0,1,OTHER,13 CONDOS - ELEVATOR APARTMENTS,2,716,1246,NaN,R4,"447 WEST 18TH STREET, PH12A",PH12A,10011.0,1.0,0.0,1.0,"10,733",1979.0,2007.0,2,R4,0,01/01/2019
1,1,OTHER,21 OFFICE BUILDINGS,4,812,68,NaN,O5,144 WEST 37TH STREET,NaN,10018.0,0.0,6.0,6.0,"2,962",15435.0,1920.0,4,O5,0,01/01/2019
2,1,OTHER,21 OFFICE BUILDINGS,4,839,69,NaN,O5,40 WEST 38TH STREET,NaN,10018.0,0.0,7.0,7.0,"2,074",11332.0,1930.0,4,O5,0,01/01/2019
3,1,OTHER,13 CONDOS - ELEVATOR APARTMENTS,2,592,1041,NaN,R4,"1 SHERIDAN SQUARE, 8C",8C,10014.0,1.0,0.0,1.0,0,500.0,0.0,2,R4,0,01/01/2019
4,1,UPPER EAST SIDE (59-79),15 CONDOS - 2-10 UNIT RESIDENTIAL,2C,1379,1402,NaN,R1,"20 EAST 65TH STREET, B",B,10065.0,1.0,0.0,1.0,0,6406.0,0.0,2,R1,0,01/01/2019


In [390]:
df.describe()

,BLOCK,LOT,EASE-MENT,ZIP_CODE,RESIDENTIAL_UNITS,COMMERCIAL_UNITS,TOTAL_UNITS,GROSS_SQUARE_FEET,YEAR_BUILT,TAX_CLASS_AT_TIME_OF_SALE,SALE_PRICE
count,23040.000000,23040.000000,0.0,23039.000000,23039.000000,23039.000000,23039.000000,2.303900e+04,23005.000000,23040.000000,2.304000e+04
mean,4459.116493,353.948481,NaN,10791.186987,1.827423,0.286514,2.320891,3.508555e+03,1819.671724,1.624479,1.328133e+06
std,3713.269887,628.025255,NaN,1103.414210,10.090432,5.778224,12.070554,2.218610e+04,488.376571,0.803898,1.025394e+07
min,1.000000,1.000000,NaN,0.000000,0.000000,-148.000000,0.000000,0.000000e+00,0.000000,1.000000,0.000000e+00
25%,1341.000000,22.000000,NaN,10306.000000,0.000000,0.000000,1.000000,4.940000e+02,1920.000000,1.000000,0.000000e+00
50%,3546.000000,49.000000,NaN,11211.000000,1.000000,0.000000,1.000000,1.356000e+03,1940.000000,1.000000,4.550000e+05
75%,6673.750000,375.000000,NaN,11360.000000,2.000000,0.000000,2.000000,2.280000e+03,1965.000000,2.000000,8.750000e+05
max,16350.000000,9057.000000,NaN,11697.000000,750.000000,570.000000,755.000000,1.303935e+06,2019.000000,4.000000,8.500000e+08


In [391]:
df.isna().sum()

BOROUGH                               0
NEIGHBORHOOD                          0
BUILDING_CLASS_CATEGORY               0
TAX_CLASS_AT_PRESENT                  1
BLOCK                                 0
LOT                                   0
EASE-MENT                         23040
BUILDING_CLASS_AT_PRESENT             1
ADDRESS                               0
APARTMENT_NUMBER                  17839
ZIP_CODE                              1
RESIDENTIAL_UNITS                     1
COMMERCIAL_UNITS                      1
TOTAL_UNITS                           1
LAND_SQUARE_FEET                     53
GROSS_SQUARE_FEET                     1
YEAR_BUILT                           35
TAX_CLASS_AT_TIME_OF_SALE             0
BUILDING_CLASS_AT_TIME_OF_SALE        0
SALE_PRICE                            0
SALE_DATE                             0
dtype: int64

In [392]:
df = df.drop(columns=['EASE-MENT','APARTMENT_NUMBER'])
df.isna().sum()

BOROUGH                            0
NEIGHBORHOOD                       0
BUILDING_CLASS_CATEGORY            0
TAX_CLASS_AT_PRESENT               1
BLOCK                              0
LOT                                0
BUILDING_CLASS_AT_PRESENT          1
ADDRESS                            0
ZIP_CODE                           1
RESIDENTIAL_UNITS                  1
COMMERCIAL_UNITS                   1
TOTAL_UNITS                        1
LAND_SQUARE_FEET                  53
GROSS_SQUARE_FEET                  1
YEAR_BUILT                        35
TAX_CLASS_AT_TIME_OF_SALE          0
BUILDING_CLASS_AT_TIME_OF_SALE     0
SALE_PRICE                         0
SALE_DATE                          0
dtype: int64

In [393]:
df.fillna(df.mean(), inplace=True)
df.isna().sum()

BOROUGH                            0
NEIGHBORHOOD                       0
BUILDING_CLASS_CATEGORY            0
TAX_CLASS_AT_PRESENT               1
BLOCK                              0
LOT                                0
BUILDING_CLASS_AT_PRESENT          1
ADDRESS                            0
ZIP_CODE                           0
RESIDENTIAL_UNITS                  0
COMMERCIAL_UNITS                   0
TOTAL_UNITS                        0
LAND_SQUARE_FEET                  53
GROSS_SQUARE_FEET                  0
YEAR_BUILT                         0
TAX_CLASS_AT_TIME_OF_SALE          0
BUILDING_CLASS_AT_TIME_OF_SALE     0
SALE_PRICE                         0
SALE_DATE                          0
dtype: int64

In [394]:
df.dropna(inplace=True)
df.isna().sum()

BOROUGH                           0
NEIGHBORHOOD                      0
BUILDING_CLASS_CATEGORY           0
TAX_CLASS_AT_PRESENT              0
BLOCK                             0
LOT                               0
BUILDING_CLASS_AT_PRESENT         0
ADDRESS                           0
ZIP_CODE                          0
RESIDENTIAL_UNITS                 0
COMMERCIAL_UNITS                  0
TOTAL_UNITS                       0
LAND_SQUARE_FEET                  0
GROSS_SQUARE_FEET                 0
YEAR_BUILT                        0
TAX_CLASS_AT_TIME_OF_SALE         0
BUILDING_CLASS_AT_TIME_OF_SALE    0
SALE_PRICE                        0
SALE_DATE                         0
dtype: int64

In [395]:
df['LAND_SQUARE_FEET'] = (
    df['LAND_SQUARE_FEET']
    .str.replace(',','')
    .str.replace('#','0')
    .astype(int)
)

df.LAND_SQUARE_FEET.describe()

# df.drop(columns='LAND_SQUARE_FEET', inplace=True)

count     22987.000000
mean       5444.546483
std       23856.916032
min           0.000000
25%           0.000000
50%        2000.000000
75%        3417.500000
max      970000.000000
Name: LAND_SQUARE_FEET, dtype: float64

In [0]:
df['GROSS_SQUARE_FEET'] = df['GROSS_SQUARE_FEET'].fillna(0).astype(int)

In [0]:
df['YEAR_BUILT'] = df['YEAR_BUILT'].fillna(0).astype(int)

In [0]:
df['ZIP_CODE'] = df['ZIP_CODE'].fillna(0).astype(int).astype(str).replace('^0$', '00000')

In [0]:
df.drop(columns='TAX_CLASS_AT_TIME_OF_SALE', inplace=True)

In [400]:
new = df['BUILDING_CLASS_CATEGORY'] == '01 ONE FAMILY DWELLINGS'
df = df[new]
print(df.shape)
df.head()

(5061, 18)


,BOROUGH,NEIGHBORHOOD,BUILDING_CLASS_CATEGORY,TAX_CLASS_AT_PRESENT,BLOCK,LOT,BUILDING_CLASS_AT_PRESENT,ADDRESS,ZIP_CODE,RESIDENTIAL_UNITS,COMMERCIAL_UNITS,TOTAL_UNITS,LAND_SQUARE_FEET,GROSS_SQUARE_FEET,YEAR_BUILT,BUILDING_CLASS_AT_TIME_OF_SALE,SALE_PRICE,SALE_DATE
7,2,OTHER,01 ONE FAMILY DWELLINGS,1,4090,37,A1,1193 SACKET AVENUE,10461,1.0,0.0,1.0,3404,1328,1925,A1,0,01/01/2019
8,2,OTHER,01 ONE FAMILY DWELLINGS,1,4120,18,A5,1215 VAN NEST AVENUE,10461,1.0,0.0,1.0,2042,1728,1935,A5,0,01/01/2019
9,2,OTHER,01 ONE FAMILY DWELLINGS,1,4120,20,A5,1211 VAN NEST AVENUE,10461,1.0,0.0,1.0,2042,1728,1935,A5,0,01/01/2019
42,3,OTHER,01 ONE FAMILY DWELLINGS,1,6809,54,A1,2601 AVENUE R,11229,1.0,0.0,1.0,3333,1262,1925,A1,0,01/01/2019
44,3,OTHER,01 ONE FAMILY DWELLINGS,1,5495,801,A9,4832 BAY PARKWAY,11230,1.0,0.0,1.0,6800,1325,1930,A9,550000,01/01/2019


In [401]:
affl = (df.SALE_PRICE <= 2000000) & (df.SALE_PRICE >= 100000)
df = df[affl]
print(df.shape)
df.head()

(3164, 18)


,BOROUGH,NEIGHBORHOOD,BUILDING_CLASS_CATEGORY,TAX_CLASS_AT_PRESENT,BLOCK,LOT,BUILDING_CLASS_AT_PRESENT,ADDRESS,ZIP_CODE,RESIDENTIAL_UNITS,COMMERCIAL_UNITS,TOTAL_UNITS,LAND_SQUARE_FEET,GROSS_SQUARE_FEET,YEAR_BUILT,BUILDING_CLASS_AT_TIME_OF_SALE,SALE_PRICE,SALE_DATE
44,3,OTHER,01 ONE FAMILY DWELLINGS,1,5495,801,A9,4832 BAY PARKWAY,11230,1.0,0.0,1.0,6800,1325,1930,A9,550000,01/01/2019
61,4,OTHER,01 ONE FAMILY DWELLINGS,1,7918,72,A1,80-23 232ND STREET,11427,1.0,0.0,1.0,4000,2001,1940,A1,200000,01/01/2019
78,2,OTHER,01 ONE FAMILY DWELLINGS,1,4210,19,A1,1260 RHINELANDER AVE,10461,1.0,0.0,1.0,3500,2043,1925,A1,810000,01/02/2019
108,3,OTHER,01 ONE FAMILY DWELLINGS,1,5212,69,A1,469 E 25TH ST,11226,1.0,0.0,1.0,4000,2680,1899,A1,125000,01/02/2019
111,3,OTHER,01 ONE FAMILY DWELLINGS,1,7930,121,A5,5521 WHITTY LANE,11203,1.0,0.0,1.0,1710,1872,1940,A5,620000,01/02/2019


In [402]:
df.drop(columns='BUILDING_CLASS_CATEGORY', inplace=True)
df.columns

Index(['BOROUGH', 'NEIGHBORHOOD', 'TAX_CLASS_AT_PRESENT', 'BLOCK', 'LOT',
       'BUILDING_CLASS_AT_PRESENT', 'ADDRESS', 'ZIP_CODE', 'RESIDENTIAL_UNITS',
       'COMMERCIAL_UNITS', 'TOTAL_UNITS', 'LAND_SQUARE_FEET',
       'GROSS_SQUARE_FEET', 'YEAR_BUILT', 'BUILDING_CLASS_AT_TIME_OF_SALE',
       'SALE_PRICE', 'SALE_DATE'],
      dtype='object')

In [403]:
train = df[df.SALE_DATE < '04/01/2019']
print(train.shape)
train.head()

(2517, 17)


,BOROUGH,NEIGHBORHOOD,TAX_CLASS_AT_PRESENT,BLOCK,LOT,BUILDING_CLASS_AT_PRESENT,ADDRESS,ZIP_CODE,RESIDENTIAL_UNITS,COMMERCIAL_UNITS,TOTAL_UNITS,LAND_SQUARE_FEET,GROSS_SQUARE_FEET,YEAR_BUILT,BUILDING_CLASS_AT_TIME_OF_SALE,SALE_PRICE,SALE_DATE
44,3,OTHER,1,5495,801,A9,4832 BAY PARKWAY,11230,1.0,0.0,1.0,6800,1325,1930,A9,550000,01/01/2019
61,4,OTHER,1,7918,72,A1,80-23 232ND STREET,11427,1.0,0.0,1.0,4000,2001,1940,A1,200000,01/01/2019
78,2,OTHER,1,4210,19,A1,1260 RHINELANDER AVE,10461,1.0,0.0,1.0,3500,2043,1925,A1,810000,01/02/2019
108,3,OTHER,1,5212,69,A1,469 E 25TH ST,11226,1.0,0.0,1.0,4000,2680,1899,A1,125000,01/02/2019
111,3,OTHER,1,7930,121,A5,5521 WHITTY LANE,11203,1.0,0.0,1.0,1710,1872,1940,A5,620000,01/02/2019


In [404]:
test = df[df.SALE_DATE >= '04/01/2019']
print(test.shape)
test.head()

(647, 17)


,BOROUGH,NEIGHBORHOOD,TAX_CLASS_AT_PRESENT,BLOCK,LOT,BUILDING_CLASS_AT_PRESENT,ADDRESS,ZIP_CODE,RESIDENTIAL_UNITS,COMMERCIAL_UNITS,TOTAL_UNITS,LAND_SQUARE_FEET,GROSS_SQUARE_FEET,YEAR_BUILT,BUILDING_CLASS_AT_TIME_OF_SALE,SALE_PRICE,SALE_DATE
18235,2,OTHER,1,5913,878,A1,4616 INDEPENDENCE AVENUE,10471,1.0,0.0,1.0,5000,2272,1930,A1,895000,04/01/2019
18239,2,OTHER,1,5488,48,A2,558 ELLSWORTH AVENUE,10465,1.0,0.0,1.0,2500,720,1935,A2,253500,04/01/2019
18244,3,OTHER,1,5936,31,A1,16 BAY RIDGE PARKWAY,11209,1.0,0.0,1.0,2880,2210,1925,A1,1300000,04/01/2019
18280,3,OTHER,1,7813,24,A5,1247 EAST 40TH STREET,11210,1.0,0.0,1.0,1305,1520,1915,A5,789000,04/01/2019
18285,3,OTHER,1,8831,160,A9,2314 PLUMB 2ND STREET,11229,1.0,0.0,1.0,1800,840,1925,A9,525000,04/01/2019


In [405]:
train.describe(exclude='number').T.sort_values(by='unique')

,count,unique,top,freq
TAX_CLASS_AT_PRESENT,2517,2,1,2486
BOROUGH,2517,5,4,1209
NEIGHBORHOOD,2517,7,OTHER,2368
BUILDING_CLASS_AT_TIME_OF_SALE,2517,11,A1,921
BUILDING_CLASS_AT_PRESENT,2517,13,A1,921
SALE_DATE,2517,68,01/31/2019,78
ZIP_CODE,2517,123,10312,115
ADDRESS,2517,2507,22-40 93RD STREET,2


In [406]:
train.groupby('BUILDING_CLASS_AT_TIME_OF_SALE')['SALE_PRICE'].describe()

,count,mean,std,min,25%,50%,75%,max
BUILDING_CLASS_AT_TIME_OF_SALE,,,,,,,,
A0,67.0,6.884526e+05,221482.552470,104000.0,572500.00,678000.0,833500.0,1200000.0
A1,921.0,6.398825e+05,312379.716631,100000.0,450000.00,570000.0,772500.0,1955000.0
A2,416.0,5.955554e+05,246879.753773,100000.0,425000.00,551000.0,782500.0,1614600.0
A3,38.0,1.199412e+06,433884.919670,211650.0,862966.75,1323560.0,1528750.0,1876000.0
A4,14.0,1.002209e+06,528373.885115,250000.0,484247.50,1126754.0,1338750.0,2000000.0
A5,781.0,5.705963e+05,220638.261466,100000.0,435000.00,532000.0,688888.0,1775000.0
A6,15.0,2.254655e+05,74753.109583,100000.0,188500.00,230000.0,261000.0,399888.0
A8,31.0,3.929000e+05,168174.803900,110000.0,288500.00,390000.0,462500.0,820000.0
A9,194.0,6.382551e+05,291090.573816,115000.0,471250.00,583500.0,760000.0,2000000.0


In [0]:
target = 'SALE_PRICE'
high_cardinality = ['BUILDING_CLASS_AT_PRESENT','SALE_DATE','ADDRESS']
features = train.columns.drop([target] + high_cardinality)

In [0]:
X_train = train[features]
y_train = train[target]
X_test = test[features]
y_test = test[target]

In [409]:
import category_encoders as ce
encoder = ce.OneHotEncoder(use_cat_names = True)
X_train = encoder.fit_transform(X_train)
X_train.columns

Index(['BOROUGH_3', 'BOROUGH_4', 'BOROUGH_2', 'BOROUGH_5', 'BOROUGH_1',
       'NEIGHBORHOOD_OTHER', 'NEIGHBORHOOD_FLUSHING-NORTH',
       'NEIGHBORHOOD_EAST NEW YORK', 'NEIGHBORHOOD_BEDFORD STUYVESANT',
       'NEIGHBORHOOD_FOREST HILLS',
       ...
       'BUILDING_CLASS_AT_TIME_OF_SALE_A1',
       'BUILDING_CLASS_AT_TIME_OF_SALE_A5',
       'BUILDING_CLASS_AT_TIME_OF_SALE_A0',
       'BUILDING_CLASS_AT_TIME_OF_SALE_A2',
       'BUILDING_CLASS_AT_TIME_OF_SALE_A3',
       'BUILDING_CLASS_AT_TIME_OF_SALE_S1',
       'BUILDING_CLASS_AT_TIME_OF_SALE_A4',
       'BUILDING_CLASS_AT_TIME_OF_SALE_A6',
       'BUILDING_CLASS_AT_TIME_OF_SALE_A8',
       'BUILDING_CLASS_AT_TIME_OF_SALE_S0'],
      dtype='object', length=156)

In [410]:
X_test = encoder.transform(X_test)
X_test.columns

Index(['BOROUGH_3', 'BOROUGH_4', 'BOROUGH_2', 'BOROUGH_5', 'BOROUGH_1',
       'NEIGHBORHOOD_OTHER', 'NEIGHBORHOOD_FLUSHING-NORTH',
       'NEIGHBORHOOD_EAST NEW YORK', 'NEIGHBORHOOD_BEDFORD STUYVESANT',
       'NEIGHBORHOOD_FOREST HILLS',
       ...
       'BUILDING_CLASS_AT_TIME_OF_SALE_A1',
       'BUILDING_CLASS_AT_TIME_OF_SALE_A5',
       'BUILDING_CLASS_AT_TIME_OF_SALE_A0',
       'BUILDING_CLASS_AT_TIME_OF_SALE_A2',
       'BUILDING_CLASS_AT_TIME_OF_SALE_A3',
       'BUILDING_CLASS_AT_TIME_OF_SALE_S1',
       'BUILDING_CLASS_AT_TIME_OF_SALE_A4',
       'BUILDING_CLASS_AT_TIME_OF_SALE_A6',
       'BUILDING_CLASS_AT_TIME_OF_SALE_A8',
       'BUILDING_CLASS_AT_TIME_OF_SALE_S0'],
      dtype='object', length=156)

In [411]:
len(X_train.columns)

156

In [412]:
from sklearn.feature_selection import SelectKBest, f_regression
selector = SelectKBest(score_func=f_regression, k=15)
X_train_select = selector.fit_transform(X_train, y_train)
X_test_select = selector.transform(X_test)
X_train_select.shape, X_test_select.shape

((2517, 15), (647, 15))

In [0]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error

In [414]:
for k in range(1, len(X_train.columns)+1):
  print(f'{k} features:')

  selector = SelectKBest(score_func=f_regression, k=k)
  X_train_select = selector.fit_transform(X_train, y_train)
  X_test_select = selector.transform(X_test)

  model = LinearRegression()
  model.fit(X_train_select, y_train)
  y_pred = model.predict(X_test_select)
  mae = mean_absolute_error(y_test, y_pred)
  print(f'Test Mean Abs. Error = ${mae:,.2f} \n')

1 features:
Test Mean Abs. Error = $185,788.22 

2 features:
Test Mean Abs. Error = $181,871.46 

3 features:
Test Mean Abs. Error = $181,602.59 

4 features:
Test Mean Abs. Error = $181,376.33 

5 features:
Test Mean Abs. Error = $175,853.70 

6 features:
Test Mean Abs. Error = $174,390.66 

7 features:
Test Mean Abs. Error = $174,021.20 

8 features:
Test Mean Abs. Error = $172,778.12 

9 features:
Test Mean Abs. Error = $172,739.68 

10 features:
Test Mean Abs. Error = $169,961.46 

11 features:
Test Mean Abs. Error = $157,213.08 

12 features:
Test Mean Abs. Error = $156,790.49 

13 features:
Test Mean Abs. Error = $156,503.09 

14 features:
Test Mean Abs. Error = $155,962.82 

15 features:
Test Mean Abs. Error = $154,389.30 

16 features:
Test Mean Abs. Error = $152,896.29 

17 features:
Test Mean Abs. Error = $154,879.77 

18 features:
Test Mean Abs. Error = $154,856.63 

19 features:
Test Mean Abs. Error = $156,132.88 

20 features:
Test Mean Abs. Error = $155,997.24 

21 featur

# According to this report, the optimal number of features to run through SelectKBest is k=89, with an
#MAE = $139,209.06

In [417]:
from sklearn.feature_selection import SelectKBest, f_regression
selector = SelectKBest(score_func=f_regression, k=89)
X_train_select = selector.fit_transform(X_train, y_train)
X_test_select = selector.transform(X_test)
X_train_select.shape, X_test_select.shape

((2517, 89), (647, 89))

In [418]:
selected_mask = selector.get_support()
all_names = X_train.columns
selected_names = all_names[selected_mask]
excluded_names = all_names[~selected_mask]

print('Features selected:')
for name in selected_names:
  print(name)

print('\n Features excluded:')
for name in excluded_names:
  print(name)

Features selected:
BOROUGH_3
BOROUGH_4
BOROUGH_2
BOROUGH_5
NEIGHBORHOOD_OTHER
NEIGHBORHOOD_FLUSHING-NORTH
NEIGHBORHOOD_EAST NEW YORK
NEIGHBORHOOD_FOREST HILLS
NEIGHBORHOOD_BOROUGH PARK
NEIGHBORHOOD_ASTORIA
TAX_CLASS_AT_PRESENT_1
TAX_CLASS_AT_PRESENT_1D
BLOCK
LOT
ZIP_CODE_11230
ZIP_CODE_11229
ZIP_CODE_11364
ZIP_CODE_11373
ZIP_CODE_11365
ZIP_CODE_11429
ZIP_CODE_11415
ZIP_CODE_11413
ZIP_CODE_11434
ZIP_CODE_10312
ZIP_CODE_11236
ZIP_CODE_11228
ZIP_CODE_11218
ZIP_CODE_11210
ZIP_CODE_11411
ZIP_CODE_11362
ZIP_CODE_11367
ZIP_CODE_11366
ZIP_CODE_11432
ZIP_CODE_10306
ZIP_CODE_10302
ZIP_CODE_10469
ZIP_CODE_11215
ZIP_CODE_11360
ZIP_CODE_11361
ZIP_CODE_11375
ZIP_CODE_11363
ZIP_CODE_11374
ZIP_CODE_11694
ZIP_CODE_11422
ZIP_CODE_11433
ZIP_CODE_11104
ZIP_CODE_11357
ZIP_CODE_10466
ZIP_CODE_10473
ZIP_CODE_11209
ZIP_CODE_11412
ZIP_CODE_10303
ZIP_CODE_10465
ZIP_CODE_11223
ZIP_CODE_11697
ZIP_CODE_11358
ZIP_CODE_11204
ZIP_CODE_11208
ZIP_CODE_11355
ZIP_CODE_10467
ZIP_CODE_10310
ZIP_CODE_10471
ZIP_CODE_11693
ZI

In [0]:
from sklearn.linear_model import RidgeCV, Ridge
import matplotlib.pyplot as plt
ridge = RidgeCV(alphas=alphas, normalize=True)

In [437]:
for alpha in [0.12, 0.14, 0.15, 0.16, 0.17]:
  print(f'Ridge Regression, alpha = {alpha}')
  model = Ridge(alpha=alpha, normalize=True)
  model.fit(X_train_select, y_train)
  y_pred = model.predict(X_test_select)

  mae = mean_absolute_error(y_test, y_pred)
  print(f'Test MAE: ${mae:,.2f}')

  # coefficients = pd.Series(model.coef_, selected_names)
  # plt.figure(figsize=(16,8))
  # coefficients.sort_values().plot.barh(color='grey')
  # plt.xlim(10,-10)
  # plt.show()

Ridge Regression, alpha = 0.12
Test MAE: $138,125.33
Ridge Regression, alpha = 0.14
Test MAE: $138,039.68
Ridge Regression, alpha = 0.15
Test MAE: $138,008.65
Ridge Regression, alpha = 0.16
Test MAE: $137,983.67
Ridge Regression, alpha = 0.17
Test MAE: $137,971.26


# Matplotlib had a little stagefright on this cell, so I muted it. I honed into an optimal alpha of around 0.15. With the same 89 features, and a ridge regression of alpha=0.15, I was able to get an MAE=$138,008.65, which is $$100 less than before ridge regression!

In [424]:
X_train_select

array([[1., 0., 0., ..., 0., 0., 0.],
       [0., 1., 0., ..., 0., 0., 0.],
       [0., 0., 1., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 1., 0., ..., 0., 0., 0.]])

# Ridge Regression

## Assignment

We're going back to our other **New York City** real estate dataset. Instead of predicting apartment rents, you'll predict property sales prices.

But not just for condos in Tribeca...

- [x] Use a subset of the data where `BUILDING_CLASS_CATEGORY` == `'01 ONE FAMILY DWELLINGS'` and the sale price was more than 100 thousand and less than 2 million.
- [x] Do train/test split. Use data from January — March 2019 to train. Use data from April 2019 to test.
- [X] Do one-hot encoding of categorical features.
- [X] Do feature selection with `SelectKBest`.
- [x] Fit a ridge regression model with multiple features. Use the `normalize=True` parameter (or do [feature scaling](https://scikit-learn.org/stable/modules/preprocessing.html) beforehand — use the scaler's `fit_transform` method with the train set, and the scaler's `transform` method with the test set)
- [x] Get mean absolute error for the test set.
- [ ] As always, commit your notebook to your fork of the GitHub repo.

The [NYC Department of Finance](https://www1.nyc.gov/site/finance/taxes/property-rolling-sales-data.page) has a glossary of property sales terms and NYC Building Class Code Descriptions. The data comes from the [NYC OpenData](https://data.cityofnewyork.us/browse?q=NYC%20calendar%20sales) portal.


## Stretch Goals

Don't worry, you aren't expected to do all these stretch goals! These are just ideas to consider and choose from.

- [ ] Add your own stretch goal(s) !
- [ ] Instead of `Ridge`, try `LinearRegression`. Depending on how many features you select, your errors will probably blow up! 💥
- [ ] Instead of `Ridge`, try [`RidgeCV`](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.RidgeCV.html).
- [ ] Learn more about feature selection:
    - ["Permutation importance"](https://www.kaggle.com/dansbecker/permutation-importance)
    - [scikit-learn's User Guide for Feature Selection](https://scikit-learn.org/stable/modules/feature_selection.html)
    - [mlxtend](http://rasbt.github.io/mlxtend/) library
    - scikit-learn-contrib libraries: [boruta_py](https://github.com/scikit-learn-contrib/boruta_py) & [stability-selection](https://github.com/scikit-learn-contrib/stability-selection)
    - [_Feature Engineering and Selection_](http://www.feat.engineering/) by Kuhn & Johnson.
- [ ] Try [statsmodels](https://www.statsmodels.org/stable/index.html) if you’re interested in more inferential statistical approach to linear regression and feature selection, looking at p values and 95% confidence intervals for the coefficients.
- [ ] Read [_An Introduction to Statistical Learning_](http://faculty.marshall.usc.edu/gareth-james/ISL/ISLR%20Seventh%20Printing.pdf), Chapters 1-3, for more math & theory, but in an accessible, readable way.
- [ ] Try [scikit-learn pipelines](https://scikit-learn.org/stable/modules/compose.html).